In [1]:
import os, duckdb

# --- Paths ---
base = r"C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\aggregator\data"
notes_path = os.path.join(base, "notes-00000.parquet")
status_path = os.path.join(base, "noteStatusHistory-00000.parquet")
output_path = os.path.join(base, "master_notes_data.parquet")

print("🚀 Creating master notes dataset by joining notes with noteStatusHistory...\n")

con = duckdb.connect()
con.execute("PRAGMA memory_limit='8GB';")

# Join notes with status history
query = f"""
COPY (
    SELECT 
        n.*,
        s.* EXCLUDE (noteId)
    FROM read_parquet('{notes_path}') n
    LEFT JOIN read_parquet('{status_path}') s 
        ON n.noteId = s.noteId
    ORDER BY n.noteId
) TO '{output_path}' (FORMAT PARQUET);
"""

print("Executing join query...")
con.execute(query)

print(f"\n✅ Done! Master notes dataset saved to:\n{output_path}")

# Verify the join
print("\n📊 Verification:")
stats = con.execute(f"""
    SELECT 
        COUNT(*) as total_notes,
        COUNT(DISTINCT noteId) as unique_notes
    FROM '{output_path}'
""").fetchdf()
print(stats.to_string(index=False))

# Get column count
print("\n📋 Schema info:")
schema = con.execute(f"DESCRIBE SELECT * FROM '{output_path}'").fetchdf()
print(f"Total columns: {len(schema)}")
print("\nFirst 10 columns:")
print(schema.head(10).to_string(index=False))

# Preview
print("\n📝 Preview (first 3 rows):")
preview = con.execute(f"SELECT * FROM '{output_path}' LIMIT 3").fetchdf()
print(f"Columns: {len(preview.columns)}, Rows: {len(preview)}")

con.close()

print("\n✅ Master notes dataset creation complete!")

🚀 Creating master notes dataset by joining notes with noteStatusHistory...

Executing join query...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


✅ Done! Master notes dataset saved to:
C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\aggregator\data\master_notes_data.parquet

📊 Verification:
 total_notes  unique_notes
     2121195       2121195

📋 Schema info:
Total columns: 51

First 10 columns:
            column_name column_type null  key default extra
                 noteId      BIGINT  YES None    None  None
noteAuthorParticipantId     VARCHAR  YES None    None  None
        createdAtMillis      BIGINT  YES None    None  None
                tweetId      BIGINT  YES None    None  None
         classification     VARCHAR  YES None    None  None
             believable     VARCHAR  YES None    None  None
                harmful     VARCHAR  YES None    None  None
   validationDifficulty     VARCHAR  YES None    None  None
        misleadingOther      BIGINT  YES None    None  None
 misleadingFactualError      BIGINT  YES None    None  None

📝 Preview (first 3 rows):
Columns: 51, Ro

In [7]:
import os, duckdb

# --- Paths ---
input_path = r"C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\aggregator\data\master_notes_data.parquet"
output_path = r"C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\user_data_aggregating\data\intermediates\user_biweekly_notes_authored.parquet"

print("🚀 Creating biweekly user aggregates from master notes data...\n")

con = duckdb.connect()
con.execute("PRAGMA memory_limit='8GB';")

# Create aggregated dataset
query = f"""
COPY (
    WITH dated_notes AS (
        SELECT
            noteAuthorParticipantId,
            to_timestamp(createdAtMillis / 1000) AS createdAt,
            -- Note content flags
            classification,
            CAST(misleadingOther AS INT) AS misleadingOther,
            CAST(misleadingFactualError AS INT) AS misleadingFactualError,
            CAST(misleadingManipulatedMedia AS INT) AS misleadingManipulatedMedia,
            CAST(misleadingOutdatedInformation AS INT) AS misleadingOutdatedInformation,
            CAST(misleadingMissingImportantContext AS INT) AS misleadingMissingImportantContext,
            CAST(misleadingUnverifiedClaimAsFact AS INT) AS misleadingUnverifiedClaimAsFact,
            CAST(misleadingSatire AS INT) AS misleadingSatire,
            CAST(notMisleadingOther AS INT) AS notMisleadingOther,
            CAST(notMisleadingFactuallyCorrect AS INT) AS notMisleadingFactuallyCorrect,
            CAST(notMisleadingOutdatedButNotWhenWritten AS INT) AS notMisleadingOutdatedButNotWhenWritten,
            CAST(notMisleadingClearlySatire AS INT) AS notMisleadingClearlySatire,
            CAST(notMisleadingPersonalOpinion AS INT) AS notMisleadingPersonalOpinion,
            CAST(trustworthySources AS INT) AS trustworthySources,
            CAST(isMediaNote AS INT) AS isMediaNote,
            -- Status information
            currentStatus,
            firstNonNMRStatus,
            mostRecentNonNMRStatus,
            lockedStatus,
            currentCoreStatus,
            currentExpansionStatus,
            currentGroupStatus,
            currentDecidedBy,
            timestampMillisOfFirstNonNMRStatus,
            timestampMillisOfCurrentStatus
        FROM read_parquet('{input_path}')
        WHERE createdAtMillis IS NOT NULL
          AND to_timestamp(createdAtMillis / 1000) >= '2023-01-01'::TIMESTAMP
    ),
    biweekly_periods AS (
        SELECT
            noteAuthorParticipantId,
            createdAt,
            -- Create 2-week periods starting from 2023-01-01
            DATE '2023-01-01' + (FLOOR(EPOCH(createdAt - TIMESTAMP '2023-01-01') / (14 * 86400)) * 14) * INTERVAL '1 day' AS period_start,
            -- All other columns
            classification,
            misleadingOther, misleadingFactualError, misleadingManipulatedMedia,
            misleadingOutdatedInformation, misleadingMissingImportantContext,
            misleadingUnverifiedClaimAsFact, misleadingSatire,
            notMisleadingOther, notMisleadingFactuallyCorrect,
            notMisleadingOutdatedButNotWhenWritten, notMisleadingClearlySatire,
            notMisleadingPersonalOpinion, trustworthySources, isMediaNote,
            currentStatus, firstNonNMRStatus, mostRecentNonNMRStatus,
            lockedStatus, currentCoreStatus, currentExpansionStatus,
            currentGroupStatus, currentDecidedBy,
            timestampMillisOfFirstNonNMRStatus, timestampMillisOfCurrentStatus
        FROM dated_notes
    )
    SELECT
        noteAuthorParticipantId,
        period_start,
        period_start + INTERVAL '13 days' AS period_end,
        
        -- Count metrics
        COUNT(*) AS total_notes_authored,
        
        -- Classification counts
        SUM(CASE WHEN classification = 'NOT_MISLEADING' THEN 1 ELSE 0 END) AS not_misleading_count,
        SUM(CASE WHEN classification = 'MISINFORMED_OR_POTENTIALLY_MISLEADING' THEN 1 ELSE 0 END) AS misleading_count,
        
        -- Misleading reason aggregates (average usage)
        ROUND(AVG(misleadingOther), 4) AS avg_misleading_other,
        ROUND(AVG(misleadingFactualError), 4) AS avg_misleading_factual_error,
        ROUND(AVG(misleadingManipulatedMedia), 4) AS avg_misleading_manipulated_media,
        ROUND(AVG(misleadingOutdatedInformation), 4) AS avg_misleading_outdated_info,
        ROUND(AVG(misleadingMissingImportantContext), 4) AS avg_misleading_missing_context,
        ROUND(AVG(misleadingUnverifiedClaimAsFact), 4) AS avg_misleading_unverified_claim,
        ROUND(AVG(misleadingSatire), 4) AS avg_misleading_satire,
        
        -- Not misleading reason aggregates (average usage)
        ROUND(AVG(notMisleadingOther), 4) AS avg_not_misleading_other,
        ROUND(AVG(notMisleadingFactuallyCorrect), 4) AS avg_not_misleading_factually_correct,
        ROUND(AVG(notMisleadingOutdatedButNotWhenWritten), 4) AS avg_not_misleading_outdated_but_not_when_written,
        ROUND(AVG(notMisleadingClearlySatire), 4) AS avg_not_misleading_clearly_satire,
        ROUND(AVG(notMisleadingPersonalOpinion), 4) AS avg_not_misleading_personal_opinion,
        
        -- Quality metrics
        ROUND(AVG(trustworthySources), 4) AS avg_trustworthy_sources,
        ROUND(AVG(isMediaNote), 4) AS avg_is_media_note,
        
        -- Status aggregates
        SUM(CASE WHEN currentStatus = 'CURRENTLY_RATED_HELPFUL' THEN 1 ELSE 0 END) AS currently_helpful_count,
        SUM(CASE WHEN currentStatus = 'CURRENTLY_RATED_NOT_HELPFUL' THEN 1 ELSE 0 END) AS currently_not_helpful_count,
        SUM(CASE WHEN currentStatus = 'NEEDS_MORE_RATINGS' THEN 1 ELSE 0 END) AS needs_more_ratings_count,
        
        -- First non-NMR status aggregates
        SUM(CASE WHEN firstNonNMRStatus = 'CURRENTLY_RATED_HELPFUL' THEN 1 ELSE 0 END) AS first_status_helpful_count,
        SUM(CASE WHEN firstNonNMRStatus = 'CURRENTLY_RATED_NOT_HELPFUL' THEN 1 ELSE 0 END) AS first_status_not_helpful_count,
        
        -- Locked status
        SUM(CASE WHEN lockedStatus IS NOT NULL AND lockedStatus != '' THEN 1 ELSE 0 END) AS locked_notes_count,
        
        -- Submodel status counts
        SUM(CASE WHEN currentCoreStatus = 'CURRENTLY_RATED_HELPFUL' THEN 1 ELSE 0 END) AS core_helpful_count,
        SUM(CASE WHEN currentExpansionStatus = 'CURRENTLY_RATED_HELPFUL' THEN 1 ELSE 0 END) AS expansion_helpful_count,
        SUM(CASE WHEN currentGroupStatus = 'CURRENTLY_RATED_HELPFUL' THEN 1 ELSE 0 END) AS group_helpful_count,
        
        -- Time to first status (average in hours)
        ROUND(AVG(CASE 
            WHEN timestampMillisOfFirstNonNMRStatus IS NOT NULL 
            THEN (timestampMillisOfFirstNonNMRStatus::BIGINT - (EPOCH(createdAt) * 1000)::BIGINT) / 3600000.0
            ELSE NULL 
        END), 2) AS avg_hours_to_first_status
        
    FROM biweekly_periods
    GROUP BY noteAuthorParticipantId, period_start
    ORDER BY noteAuthorParticipantId, period_start
) TO '{output_path}' (FORMAT PARQUET);
"""

print("Executing aggregation query...")
con.execute(query)

print(f"\n✅ Done! User biweekly notes authored aggregates saved to:\n{output_path}")

# Preview results
print("\n📊 Preview of aggregated data:")
preview = con.execute(f"""
    SELECT 
        noteAuthorParticipantId,
        period_start,
        total_notes_authored,
        not_misleading_count,
        misleading_count,
        currently_helpful_count,
        currently_not_helpful_count,
        needs_more_ratings_count,
        avg_trustworthy_sources,
        avg_hours_to_first_status
    FROM '{output_path}' 
    LIMIT 10
""").fetchdf()
print(preview.to_string(index=False))

row_count = con.execute(f"SELECT COUNT(*) FROM '{output_path}'").fetchone()[0]
print(f"\nTotal user-period combinations: {row_count:,}")

con.close()

print("\n✅ Biweekly user notes authored aggregation complete!")

🚀 Creating biweekly user aggregates from master notes data...

Executing aggregation query...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


✅ Done! User biweekly notes authored aggregates saved to:
C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\user_data_aggregating\data\intermediates\user_biweekly_notes_authored.parquet

📊 Preview of aggregated data:
                                         noteAuthorParticipantId period_start  total_notes_authored  not_misleading_count  misleading_count  currently_helpful_count  currently_not_helpful_count  needs_more_ratings_count  avg_trustworthy_sources  avg_hours_to_first_status
000045A5FA0CF004F68CBF2913506C37D540CF48522D33BFBF036AAC53FBDA8B   2023-03-26                     5                   0.0               5.0                      2.0                          0.0                       3.0                      1.0                       5.25
000045A5FA0CF004F68CBF2913506C37D540CF48522D33BFBF036AAC53FBDA8B   2023-05-07                     2                   1.0               1.0                      0.0                          0.0   

In [9]:
import os, duckdb

# --- Paths ---
base = r"C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\aggregator\data"
notes_authored_path = os.path.join(base, "user_biweekly_notes_authored.parquet")
ratings_aggregates_path = os.path.join(base, "user_biweekly_aggregates.parquet")
output_path = os.path.join(base, "user_period_master.parquet")

print("🚀 Creating comprehensive user-period master dataset via full outer join...\n")

con = duckdb.connect()
con.execute("PRAGMA memory_limit='8GB';")

# Perform full outer join to combine notes authored and ratings data
query = f"""
COPY (
    SELECT 
        COALESCE(n.noteAuthorParticipantId, r.raterParticipantId) AS userId,
        COALESCE(n.period_start, r.period_start) AS period_start,
        COALESCE(n.period_end, r.period_end) AS period_end,
        
        -- Notes authored columns
        n.total_notes_authored,
        n.not_misleading_count,
        n.misleading_count,
        n.avg_misleading_other,
        n.avg_misleading_factual_error,
        n.avg_misleading_manipulated_media,
        n.avg_misleading_outdated_info,
        n.avg_misleading_missing_context,
        n.avg_misleading_unverified_claim,
        n.avg_misleading_satire,
        n.avg_not_misleading_other,
        n.avg_not_misleading_factually_correct,
        n.avg_not_misleading_outdated_but_not_when_written,
        n.avg_not_misleading_clearly_satire,
        n.avg_not_misleading_personal_opinion,
        n.avg_trustworthy_sources,
        n.avg_is_media_note,
        n.currently_helpful_count,
        n.currently_not_helpful_count,
        n.needs_more_ratings_count,
        n.first_status_helpful_count,
        n.first_status_not_helpful_count,
        n.locked_notes_count,
        n.core_helpful_count,
        n.expansion_helpful_count,
        n.group_helpful_count,
        n.avg_hours_to_first_status,
        
        -- Ratings aggregates columns
        r.total_ratings,
        r.unique_notes_rated,
        r.helpful_count,
        r.not_helpful_count,
        r.unknown_count,
        r.helpful_ratio,
        r.not_helpful_ratio,
        r.avg_core_note_intercept,
        r.min_core_note_intercept,
        r.max_core_note_intercept,
        r.stddev_core_note_intercept,
        r.non_null_intercept_count,
        r.avg_core_note_factor1,
        r.min_core_note_factor1,
        r.max_core_note_factor1,
        r.stddev_core_note_factor1,
        r.non_null_factor1_count
        
    FROM read_parquet('{notes_authored_path}') n
    FULL OUTER JOIN read_parquet('{ratings_aggregates_path}') r
        ON n.noteAuthorParticipantId = r.raterParticipantId
        AND n.period_start = r.period_start
    ORDER BY userId, period_start
) TO '{output_path}' (FORMAT PARQUET);
"""

print("Executing full outer join query...")
con.execute(query)

print(f"\n✅ Done! User-period master dataset saved to:\n{output_path}")

# Verification stats
print("\n📊 Verification:")
stats = con.execute(f"""
    SELECT 
        COUNT(*) as total_user_periods,
        COUNT(DISTINCT userId) as unique_users,
        COUNT(total_notes_authored) as periods_with_notes_authored,
        COUNT(total_ratings) as periods_with_ratings,
        COUNT(CASE WHEN total_notes_authored IS NOT NULL AND total_ratings IS NOT NULL THEN 1 END) as periods_with_both,
        MIN(period_start) as earliest_period,
        MAX(period_start) as latest_period
    FROM '{output_path}'
""").fetchdf()
print(stats.to_string(index=False))

# Preview
print("\n📝 Preview of master dataset:")
preview = con.execute(f"""
    SELECT 
        userId,
        period_start,
        total_notes_authored,
        total_ratings,
        not_misleading_count,
        misleading_count,
        helpful_count,
        not_helpful_count,
        avg_core_note_intercept,
        avg_core_note_factor1
    FROM '{output_path}'
    LIMIT 10
""").fetchdf()
print(preview.to_string(index=False))

con.close()

print("\n✅ User-period master dataset creation complete!")

🚀 Creating comprehensive user-period master dataset via full outer join...

Executing full outer join query...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


✅ Done! User-period master dataset saved to:
C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\aggregator\data\user_period_master.parquet

📊 Verification:
 total_user_periods  unique_users  periods_with_notes_authored  periods_with_ratings  periods_with_both earliest_period latest_period
           21020118       1279178                      1030703              20972455             983040      2023-01-01    2025-09-21

📝 Preview of master dataset:
                                                          userId period_start  total_notes_authored  total_ratings  not_misleading_count  misleading_count  helpful_count  not_helpful_count  avg_core_note_intercept  avg_core_note_factor1
0000010BB832A9CFDF102BF7B66896FA987C80FBB61EF6C4B04D875B85C07BD8   2023-12-31                  <NA>              1                   NaN               NaN            1.0                0.0                   0.3825                -0.5013
0000010BB832A9CFDF102BF7B66896

In [10]:
import os, duckdb

# --- Paths ---
base = r"C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\aggregator\data"
master_path = os.path.join(base, "user_period_master.parquet")
helpful_vs_not_path = os.path.join(base, "user_biweekly_helpful_vs_not.parquet")
output_path = os.path.join(base, "user_period_master_final.parquet")

print("🚀 Final enrichment: Adding helpful vs not helpful detailed analysis...\n")

con = duckdb.connect()
con.execute("PRAGMA memory_limit='8GB';")

# Join and enrich the master dataset with helpful vs not helpful columns
query = f"""
COPY (
    SELECT 
        m.*,
        -- Add helpful vs not helpful detailed columns (excluding duplicates)
        h.total_ratings AS total_ratings_hvn,
        h.unique_notes_rated AS unique_notes_rated_hvn,
        h.avg_intercept_helpful,
        h.avg_factor1_helpful,
        h.stddev_intercept_helpful,
        h.stddev_factor1_helpful,
        h.helpful_with_scores,
        h.avg_intercept_not_helpful,
        h.avg_factor1_not_helpful,
        h.stddev_intercept_not_helpful,
        h.stddev_factor1_not_helpful,
        h.not_helpful_with_scores,
        h.intercept_diff_helpful_minus_not,
        h.factor1_diff_helpful_minus_not,
        h.unknown_count AS unknown_count_hvn
        
    FROM read_parquet('{master_path}') m
    LEFT JOIN read_parquet('{helpful_vs_not_path}') h 
        ON m.userId = h.raterParticipantId 
        AND m.period_start = h.period_start
    ORDER BY m.userId, m.period_start
) TO '{output_path}' (FORMAT PARQUET);
"""

print("Executing final enrichment query...")
con.execute(query)

print(f"\n✅ Done! Final enriched master dataset saved to:\n{output_path}")

# Verification stats
print("\n📊 Verification:")
stats = con.execute(f"""
    SELECT 
        COUNT(*) as total_user_periods,
        COUNT(DISTINCT userId) as unique_users,
        COUNT(total_notes_authored) as periods_with_notes_authored,
        COUNT(total_ratings) as periods_with_ratings,
        COUNT(avg_intercept_helpful) as periods_with_hvn_data,
        COUNT(CASE WHEN total_notes_authored IS NOT NULL AND total_ratings IS NOT NULL THEN 1 END) as periods_with_both_activities,
        MIN(period_start) as earliest_period,
        MAX(period_start) as latest_period
    FROM '{output_path}'
""").fetchdf()
print(stats.to_string(index=False))

# Get column count
schema = con.execute(f"DESCRIBE SELECT * FROM '{output_path}'").fetchdf()
print(f"\nTotal columns in final dataset: {len(schema)}")

# Preview
print("\n📝 Preview of final enriched dataset:")
preview = con.execute(f"""
    SELECT 
        userId,
        period_start,
        total_notes_authored,
        total_ratings,
        helpful_count,
        not_helpful_count,
        avg_intercept_helpful,
        avg_intercept_not_helpful,
        intercept_diff_helpful_minus_not,
        avg_core_note_intercept,
        avg_core_note_factor1
    FROM '{output_path}'
    WHERE avg_intercept_helpful IS NOT NULL
    LIMIT 10
""").fetchdf()
print(preview.to_string(index=False))

con.close()

print("\n✅ Final user-period master dataset creation complete!")
print(f"📁 Final file: {output_path}")

🚀 Final enrichment: Adding helpful vs not helpful detailed analysis...

Executing final enrichment query...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


✅ Done! Final enriched master dataset saved to:
C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\aggregator\data\user_period_master_final.parquet

📊 Verification:
 total_user_periods  unique_users  periods_with_notes_authored  periods_with_ratings  periods_with_hvn_data  periods_with_both_activities earliest_period latest_period
           21020118       1279178                      1030703              20972455               16423919                        983040      2023-01-01    2025-09-21

Total columns in final dataset: 62

📝 Preview of final enriched dataset:
                                                          userId period_start  total_notes_authored  total_ratings  helpful_count  not_helpful_count  avg_intercept_helpful  avg_intercept_not_helpful  intercept_diff_helpful_minus_not  avg_core_note_intercept  avg_core_note_factor1
0000010BB832A9CFDF102BF7B66896FA987C80FBB61EF6C4B04D875B85C07BD8   2023-12-31                  <NA>   

In [11]:
import os, duckdb

# --- Paths ---
base = r"C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\aggregator\data"
requests_path = os.path.join(base, "noteRequests-00000.parquet")
output_path = os.path.join(base, "user_biweekly_note_requests.parquet")

print("🚀 Creating biweekly user aggregates for note requests...\n")

con = duckdb.connect()
con.execute("PRAGMA memory_limit='8GB';")

# Create aggregated dataset for note requests
query = f"""
COPY (
    WITH dated_requests AS (
        SELECT
            userId,
            tweetId,
            to_timestamp(createdAtMillis / 1000) AS createdAt,
            sourceLink,
            CASE WHEN sourceLink IS NOT NULL AND sourceLink != '' THEN 1 ELSE 0 END AS has_source_link
        FROM read_parquet('{requests_path}')
        WHERE createdAtMillis IS NOT NULL
          AND to_timestamp(createdAtMillis / 1000) >= '2023-01-01'::TIMESTAMP
    ),
    biweekly_periods AS (
        SELECT
            userId,
            tweetId,
            createdAt,
            has_source_link,
            -- Create 2-week periods starting from 2023-01-01
            DATE '2023-01-01' + (FLOOR(EPOCH(createdAt - TIMESTAMP '2023-01-01') / (14 * 86400)) * 14) * INTERVAL '1 day' AS period_start
        FROM dated_requests
    )
    SELECT
        userId,
        period_start,
        period_start + INTERVAL '13 days' AS period_end,
        
        -- Count metrics
        COUNT(*) AS total_requests,
        COUNT(DISTINCT tweetId) AS unique_tweets_requested,
        
        -- Source link usage
        SUM(has_source_link) AS requests_with_source_link,
        ROUND(AVG(has_source_link), 4) AS source_link_ratio
        
    FROM biweekly_periods
    GROUP BY userId, period_start
    ORDER BY userId, period_start
) TO '{output_path}' (FORMAT PARQUET);
"""

print("Executing aggregation query for note requests...")
con.execute(query)

print(f"\n✅ Done! User biweekly note requests aggregates saved to:\n{output_path}")

# Preview results
print("\n📊 Preview of note requests aggregated data:")
preview = con.execute(f"""
    SELECT 
        userId,
        period_start,
        total_requests,
        unique_tweets_requested,
        requests_with_source_link,
        source_link_ratio
    FROM '{output_path}' 
    LIMIT 10
""").fetchdf()
print(preview.to_string(index=False))

row_count = con.execute(f"SELECT COUNT(*) FROM '{output_path}'").fetchone()[0]
print(f"\nTotal user-period combinations: {row_count:,}")

# Get summary stats
summary = con.execute(f"""
    SELECT 
        COUNT(DISTINCT userId) as unique_requesters,
        SUM(total_requests) as total_requests_overall,
        ROUND(AVG(total_requests), 2) as avg_requests_per_user_period,
        ROUND(AVG(source_link_ratio), 4) as overall_source_link_usage_ratio
    FROM '{output_path}'
""").fetchdf()
print("\n📈 Overall Summary:")
print(summary.to_string(index=False))

con.close()

print("\n✅ Biweekly user note requests aggregation complete!")

🚀 Creating biweekly user aggregates for note requests...

Executing aggregation query for note requests...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


✅ Done! User biweekly note requests aggregates saved to:
C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\aggregator\data\user_biweekly_note_requests.parquet

📊 Preview of note requests aggregated data:
                                                          userId period_start  total_requests  unique_tweets_requested  requests_with_source_link  source_link_ratio
000004CC6A9EA228A4D367C463A49BA1A75C2B4F1FE1CB600D0076866DED7A5C   2024-08-25               1                        1                        0.0                0.0
00000D07B9A6256C0D44099CA726D30D838172E69678C05F331FC8E19F370B39   2025-04-20               1                        1                        0.0                0.0
000017720184E7316E75A7875EB214AA0AC36A8CA70B274428F6F1D0AB1A48EE   2025-06-29               1                        1                        1.0                1.0
000017720184E7316E75A7875EB214AA0AC36A8CA70B274428F6F1D0AB1A48EE   2025-07-13               1

In [12]:
import os, duckdb

# --- Paths ---
base = r"C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\aggregator\data"
master_final_path = os.path.join(base, "user_period_master_final.parquet")
requests_path = os.path.join(base, "user_biweekly_note_requests.parquet")
output_path = os.path.join(base, "user_period_master_complete.parquet")

print("🚀 Final join: Adding note requests data to create complete master dataset...\n")

con = duckdb.connect()
con.execute("PRAGMA memory_limit='8GB';")

# Join and add note requests columns
query = f"""
COPY (
    SELECT 
        m.*,
        -- Add note requests columns
        r.total_requests,
        r.unique_tweets_requested,
        r.requests_with_source_link,
        r.source_link_ratio
        
    FROM read_parquet('{master_final_path}') m
    LEFT JOIN read_parquet('{requests_path}') r 
        ON m.userId = r.userId 
        AND m.period_start = r.period_start
    ORDER BY m.userId, m.period_start
) TO '{output_path}' (FORMAT PARQUET);
"""

print("Executing final join with note requests...")
con.execute(query)

print(f"\n✅ Done! Complete master dataset saved to:\n{output_path}")

# Verification stats
print("\n📊 Final Verification:")
stats = con.execute(f"""
    SELECT 
        COUNT(*) as total_user_periods,
        COUNT(DISTINCT userId) as unique_users,
        COUNT(total_notes_authored) as periods_with_notes_authored,
        COUNT(total_ratings) as periods_with_ratings,
        COUNT(total_requests) as periods_with_requests,
        COUNT(avg_intercept_helpful) as periods_with_hvn_data,
        COUNT(CASE WHEN total_notes_authored IS NOT NULL AND total_ratings IS NOT NULL AND total_requests IS NOT NULL THEN 1 END) as periods_with_all_activities,
        MIN(period_start) as earliest_period,
        MAX(period_start) as latest_period
    FROM '{output_path}'
""").fetchdf()
print(stats.to_string(index=False))

# Get column count
schema = con.execute(f"DESCRIBE SELECT * FROM '{output_path}'").fetchdf()
print(f"\nTotal columns in complete dataset: {len(schema)}")

# Preview
print("\n📝 Preview of complete final dataset:")
preview = con.execute(f"""
    SELECT 
        userId,
        period_start,
        total_notes_authored,
        total_ratings,
        total_requests,
        helpful_count,
        not_helpful_count,
        avg_intercept_helpful,
        avg_intercept_not_helpful,
        avg_core_note_intercept,
        unique_tweets_requested,
        source_link_ratio
    FROM '{output_path}'
    WHERE total_requests IS NOT NULL
    LIMIT 10
""").fetchdf()
print(preview.to_string(index=False))

con.close()

print("\n" + "=" * 100)
print("🎉 COMPLETE USER-PERIOD MASTER DATASET CREATION FINISHED! 🎉")
print("=" * 100)
print(f"📁 Final complete file: {output_path}")
print("\n📋 Dataset includes:")
print("   • Notes authored by users (with classifications, statuses, quality metrics)")
print("   • Ratings given by users (with helpful/not helpful analysis)")
print("   • Note intercept & factor analysis (helpful vs not helpful breakdown)")
print("   • Note requests submitted by users")
print("   • All data aggregated by user and biweekly period from 2023-01-01 onwards")
print("=" * 100)

🚀 Final join: Adding note requests data to create complete master dataset...

Executing final join with note requests...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


✅ Done! Complete master dataset saved to:
C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\aggregator\data\user_period_master_complete.parquet

📊 Final Verification:
 total_user_periods  unique_users  periods_with_notes_authored  periods_with_ratings  periods_with_requests  periods_with_hvn_data  periods_with_all_activities earliest_period latest_period
           21020118       1279178                      1030703              20972455                 352918               16423919                        38294      2023-01-01    2025-09-21

Total columns in complete dataset: 66

📝 Preview of complete final dataset:
                                                          userId period_start  total_notes_authored  total_ratings  total_requests  helpful_count  not_helpful_count  avg_intercept_helpful  avg_intercept_not_helpful  avg_core_note_intercept  unique_tweets_requested  source_link_ratio
00002C7FD6E0080A69D0AB879C3D9BB704BEFCC078AD400FA

In [2]:
import os, duckdb

# --- Paths ---
base = r"C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\aggregator\data"
input_path = os.path.join(base, "user_period_master_complete.parquet")
output_path = os.path.join(base, "user_period_master_clean.parquet")

print("🚀 Removing duplicate columns from master dataset...\n")

con = duckdb.connect()
con.execute("PRAGMA memory_limit='8GB';")

# Drop redundant columns (total_ratings_hvn, unique_notes_rated_hvn, unknown_count_hvn, unique_notes_rated)
query = f"""
COPY (
    SELECT 
        * EXCLUDE (total_ratings_hvn, unique_notes_rated_hvn, unknown_count_hvn, unique_notes_rated)
    FROM read_parquet('{input_path}')
    ORDER BY userId, period_start
) TO '{output_path}' (FORMAT PARQUET);
"""

print("Executing column removal query...")
con.execute(query)

print(f"\n✅ Done! Clean master dataset saved to:\n{output_path}")

# Get column counts
original_schema = con.execute(f"DESCRIBE SELECT * FROM '{input_path}'").fetchdf()
clean_schema = con.execute(f"DESCRIBE SELECT * FROM '{output_path}'").fetchdf()

print("\n📊 Column Comparison:")
print(f"Original columns: {len(original_schema)}")
print(f"Clean columns: {len(clean_schema)}")
print(f"Removed: {len(original_schema) - len(clean_schema)} duplicate columns")

print("\n📋 Removed columns:")
print("  • total_ratings_hvn (duplicate of total_ratings)")
print("  • unique_notes_rated_hvn (duplicate of unique_notes_rated)")
print("  • unknown_count_hvn (duplicate of unknown_count)")
print("  • unique_notes_rated (redundant - equals total_ratings since users can't rate notes twice)")

# Preview
print("\n📝 Preview of clean dataset:")
preview = con.execute(f"""
    SELECT 
        userId,
        period_start,
        total_notes_authored,
        total_ratings,
        total_requests,
        helpful_count,
        not_helpful_count,
        avg_intercept_helpful,
        avg_intercept_not_helpful,
        intercept_diff_helpful_minus_not,
        avg_core_note_intercept
    FROM '{output_path}'
    LIMIT 10
""").fetchdf()
print(preview.to_string(index=False))

con.close()

print("\n✅ Clean dataset creation complete!")
print(f"📁 Final clean file: {output_path}")

🚀 Removing duplicate columns from master dataset...

Executing column removal query...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


✅ Done! Clean master dataset saved to:
C:\Users\wongb\twitter-community-notes-time-series\twitter-community-notes-user-time-series\aggregator\data\user_period_master_clean.parquet

📊 Column Comparison:
Original columns: 66
Clean columns: 62
Removed: 4 duplicate columns

📋 Removed columns:
  • total_ratings_hvn (duplicate of total_ratings)
  • unique_notes_rated_hvn (duplicate of unique_notes_rated)
  • unknown_count_hvn (duplicate of unknown_count)
  • unique_notes_rated (redundant - equals total_ratings since users can't rate notes twice)

📝 Preview of clean dataset:
                                                          userId period_start  total_notes_authored  total_ratings  total_requests  helpful_count  not_helpful_count  avg_intercept_helpful  avg_intercept_not_helpful  intercept_diff_helpful_minus_not  avg_core_note_intercept
0000010BB832A9CFDF102BF7B66896FA987C80FBB61EF6C4B04D875B85C07BD8   2023-12-31                  <NA>              1            <NA>            1.0     